In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import *
import xgboost as xgb
import optuna
from sklearn.ensemble import VotingRegressor
from lightgbm import LGBMRegressor, early_stopping
from xgboost import XGBRegressor
import pickle

from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [8]:
df = pd.read_csv("trainable_df.csv")

In [9]:
df

,Health Conscious Level,SCALER_Annual_Income,SCALER_Credit_Score,SCALER_IsNull_Annual_Income,SCALER_Money_Handling_Level,SCALER_Money_Handling_Level1,SCALER_Money_Per_Head,SCALER_Growth,SCALER_Credit_by_Score,SCALER_Determinstic,SCALER_Growth1,SCALER_Feedback1,SCALER_Previous_Claims_MEDIAN_Premium_Amount,SCALER_IsNull_Health_Score,SCALER_Previous_Claims_MEAN_Premium_Amount,SCALER_Previous_Claims,SCALER_Previous_Claims_STD_Premium_Amount,SCALER_IsNull_Customer_Feedback,SCALER_Previous_Claims_MAX_Premium_Amount,SCALER_Feedback3,SCALER_IsNull_Previous_Claims,SCALER_IsNull_Marital_Status,SCALER_Health_Score,SCALER_Health_Risk_Score,SCALER_Feedback2,SCALER_CreditInsurance,SCALER_Sin_Year,SCALER_IsNull_Credit_Score,SCALER_Health_Age_Interaction,SCALER_ENCODED_Policy_Start_Date_-_Year,SCALER_ENCODED_Policy_Start_Date_-_Quarter,SCALER_Feedback4,SCALER_IsNull_Number_of_Dependents,SCALER_IsNull_Occupation,SCALER_Health_Conscious_Level1,SCALER_Sin_Month,SCALER_Policy_Start_Date_-_Month,SCALER_Health_Conscious_Level,SCALER_Health_Conscious_Level_MEAN_Premium_Amount,SCALER_Health_Conscious_Level_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_MEAN_Premium_Amount,SCALER_Number_of_Dependents_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_STD_Premium_Amount,SCALER_Insurance_Duration_MEAN_Premium_Amount,SCALER_Insurance_Duration_MEDIAN_Premium_Amount,SCALER_Health_Conscious_Level_MAX_Premium_Amount,SCALER_Credit_Health_Score,SCALER_Occupation_MEDIAN_Premium_Amount,SCALER_Occupation_MAX_Premium_Amount,SCALER_Occupation_MEAN_Premium_Amount,SCALER_Previous_Claims_MIN_Premium_Amount,SCALER_Insurance_Duration_MAX_Premium_Amount,SCALER_ENCODED_Occupation_Self-Employed,SCALER_Age,SCALER_Insurance_Duration_STD_Premium_Amount,Premium Amount
0,4,-0.391362,-0.978541,0.0,-0.498737,-0.174529,-0.069736,-0.323087,-1.087379,-0.065465,-0.257215,-0.405037,52.0,0.0,57.590036,1.0,30.472002,0.0,-4.5,0.00,0.0,0.0,-0.122550,0.122550,-0.725022,-0.347664,0.000000,0.0,-0.600756,0.333333,0.7,-0.422990,0.0,0.0,-0.014009,-1.469576e-15,1.2,0.023383,0.0,0.095653,0.000,0.540228,-0.739699,0.00,-0.500000,-0.502009,-0.333333,0.4,-0.619544,0.0,0.222222,1.0,-0.956522,-2.154674,0.000000,2869.0
1,2,0.199691,0.403433,0.0,0.377435,0.111549,-0.047019,0.468764,0.556634,0.216862,0.044644,0.231799,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,-0.520242,0.520242,0.173298,-0.524112,0.000000,0.0,-0.389580,0.333333,0.5,-0.266721,0.0,1.0,-0.384095,0.000000e+00,0.0,-1.208313,-1.8,0.000000,0.125,0.000000,0.458999,1.50,0.833333,-0.289535,0.000000,0.0,0.000000,0.0,0.333333,0.0,-0.086957,-0.781814,0.469401,1483.0
2,4,0.033653,0.137339,0.0,0.098729,0.032690,-0.137176,-0.264926,0.355987,0.516638,0.865042,0.698337,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.50,0.0,0.0,1.268081,-1.268081,1.181256,-0.334206,0.000000,1.0,0.177849,0.333333,0.6,2.616350,0.0,0.0,0.136874,-7.347881e-16,0.6,0.023383,0.0,0.000000,0.125,0.000000,-0.557842,0.00,-0.500000,1.394451,-0.333333,0.4,-0.619544,0.0,0.333333,1.0,-0.782609,0.000000,0.000000,567.0
3,3,3.210485,-1.000000,0.0,1.821672,5.345443,2.640256,2.462508,-0.501618,6.137915,3.337008,1.169594,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,-0.25,0.0,0.0,-0.782291,0.782291,-0.729443,-0.905794,1.000000,0.0,-0.837856,0.666667,0.9,-0.636344,0.0,1.0,-0.280236,0.000000e+00,0.0,-0.976617,-0.8,0.631238,0.250,3.658057,-1.483870,-2.75,0.500000,-0.891093,-0.333333,0.4,-0.619544,0.0,-0.666667,1.0,-0.869565,-0.132030,0.000000,765.0
4,3,0.417568,-0.008584,0.0,0.460353,0.428771,1.247990,0.302523,0.245955,1.288822,0.550003,-0.051394,0.0,0.0,-1.000000,-1.0,-1.000000,0.0,1.0,-0.50,0.0,0.0,-0.248305,0.248305,-0.525199,-0.148785,0.753709,0.0,-0.602512,-0.333333,-0.1,-0.463658,0.0,0.0,-0.301238,-1.469576e-15,1.2,-0.976617,-0.8,0.095653,0.000,0.540228,0.032506,0.00,0.500000,-0.167337,-0.333333,0.4,-0.619544,0.0,-0.333333,1.0,-0.869565,0.218186,0.000000,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

#
---
#

# Just Checking!

In [10]:
def return_splits(ddf, feature_name, target_name):
    return [ddf[ddf[feature_name] == i][target_name] for i in ddf[feature_name].unique()]

def give_stats_analysis(df, target_column_name):
    ddf = df.copy()
    ddf = ddf.dropna()

    features = []
    tests = []
    stats = []
    pvals = []
    verdict = []
    count = 0

    target = ddf[target_column_name]
    for i in ddf.columns:
        features.append(i)
        feature = ddf[i]
        
        if (feature.dtype == "O" and (target.dtype == "float" or target.dtype == "int")) or (target.dtype == "O" and (feature.dtype == "float" or feature.dtype == "int")):
            stat, pval, *_ = kruskal(*return_splits(ddf, feature.name, target.name))
            tests.append("Kruskal-Wallis")
            stats.append(stat)
            pvals.append(pval)
            
        
        elif (feature.dtype == "float" or feature.dtype == "int") and (target.dtype == "float" or target.dtype == "int"):
            stat, pval, *_ = spearmanr(feature, target)
            tests.append("SpearmanR")
            stats.append(stat)
            pvals.append(pval)

        elif feature.dtype == "O" and target.dtype == "O":
            stat, pval, *_ = chi2_contingency(pd.crosstab(feature, target))
            tests.append("Chi-Square")
            stats.append(stat)
            pvals.append(pval)
        
        else:
            tests.append(np.nan)
            stats.append(np.nan)
            pvals.append(np.nan)
        
        if pval <= 0.05:
            verdict.append("There is Relationship")
        else:
            verdict.append("There is NO Relationship")

        print(f"{feature.name} ■■■ {target_column_name}".ljust(50, "-")+"✅")
    
    return pd.DataFrame({
        "Feature" : features,
        "Target" : [target_column_name]*ddf.shape[1],
        "Statistic Test" : tests,
        "Test Statistic" : stats,
        "P-Value" : pvals,
        "Verdict" : verdict
    }).sort_values(by="P-Value")

In [11]:
give_stats_analysis(df, "Premium Amount")

Health Conscious Level ■■■ Premium Amount---------✅
SCALER_Annual_Income ■■■ Premium Amount-----------✅
SCALER_Credit_Score ■■■ Premium Amount------------✅
SCALER_IsNull_Annual_Income ■■■ Premium Amount----✅
SCALER_Money_Handling_Level ■■■ Premium Amount----✅
SCALER_Money_Handling_Level1 ■■■ Premium Amount---✅
SCALER_Money_Per_Head ■■■ Premium Amount----------✅
SCALER_Growth ■■■ Premium Amount------------------✅
SCALER_Credit_by_Score ■■■ Premium Amount---------✅
SCALER_Determinstic ■■■ Premium Amount------------✅
SCALER_Growth1 ■■■ Premium Amount-----------------✅
SCALER_Feedback1 ■■■ Premium Amount---------------✅
SCALER_Previous_Claims_MEDIAN_Premium_Amount ■■■ Premium Amount✅
SCALER_IsNull_Health_Score ■■■ Premium Amount-----✅
SCALER_Previous_Claims_MEAN_Premium_Amount ■■■ Premium Amount✅
SCALER_Previous_Claims ■■■ Premium Amount---------✅
SCALER_Previous_Claims_STD_Premium_Amount ■■■ Premium Amount✅
SCALER_IsNull_Customer_Feedback ■■■ Premium Amount✅
SCALER_Previous_Claims_MAX_Pre

,Feature,Target,Statistic Test,Test Statistic,P-Value,Verdict
1,SCALER_Annual_Income,Premium Amount,SpearmanR,-0.061831,0.000000e+00,There is Relationship
2,SCALER_Credit_Score,Premium Amount,SpearmanR,-0.036687,0.000000e+00,There is Relationship
3,SCALER_IsNull_Annual_Income,Premium Amount,SpearmanR,-0.065399,0.000000e+00,There is Relationship
4,SCALER_Money_Handling_Level,Premium Amount,SpearmanR,-0.072097,0.000000e+00,There is Relationship
6,SCALER_Money_Per_Head,Premium Amount,SpearmanR,-0.053422,0.000000e+00,There is Relationship
5,SCALER_Money_Handling_Level1,Premium Amount,SpearmanR,-0.048668,0.000000e+00,There is Relationship
7,SCALER_Growth,Premium Amount,SpearmanR,-0.055000,0.000000e+00,There is Relationship
8,SCALER_Credit_by_Score,Premium Amount,SpearmanR,-0.054850,0.000000e+00,There is Relationship
12,SCALER_Previous_Claims_MEDIAN_Premium_Amount,Premium Amount,SpearmanR,0.037268,0.000000e+00,There is Relationship
9,SCALER_Determinstic,Premium Amount,SpearmanR,-0.056869,0.000000e+00,There is Relationship


In [12]:
df.isnull().sum()

Health Conscious Level                                      0
SCALER_Annual_Income                                        0
SCALER_Credit_Score                                         0
SCALER_IsNull_Annual_Income                                 0
SCALER_Money_Handling_Level                                 0
SCALER_Money_Handling_Level1                                0
SCALER_Money_Per_Head                                       0
SCALER_Growth                                               0
SCALER_Credit_by_Score                                      0
SCALER_Determinstic                                         0
SCALER_Growth1                                              0
SCALER_Feedback1                                            0
SCALER_Previous_Claims_MEDIAN_Premium_Amount                0
SCALER_IsNull_Health_Score                                  0
SCALER_Previous_Claims_MEAN_Premium_Amount                  0
SCALER_Previous_Claims                                      0
SCALER_P

#
---
#

# Removing non related columns

In [13]:
# df.drop(columns=["SCALER_ENCODED_Marital_Status_Married", "SCALER_ENCODED_Customer_Feedback", "SCALER_PC2_Meaningless_df", "SCALER_PC1_Meaningless_df", "SCALER_ENCODED_Occupation_Unemployed"], inplace=True)

#
---
#

# Spliting Data

In [14]:
train = df.iloc[:1200000, :]
test = df.iloc[1200000:, :]

train.shape, test.shape

((1200000, 56), (800000, 56))

In [15]:
X = train.drop(columns="Premium Amount")
Y = train["Premium Amount"]
Y_log = np.log1p(Y)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_train, x_validate, y_train, y_validate = train_test_split(X, Y, test_size=10000)

In [18]:
x_validate.shape

(10000, 55)

In [19]:
test.drop(columns="Premium Amount", inplace=True)

In [20]:
test.shape

(800000, 55)

In [21]:
x_validate.head(3)

,Health Conscious Level,SCALER_Annual_Income,SCALER_Credit_Score,SCALER_IsNull_Annual_Income,SCALER_Money_Handling_Level,SCALER_Money_Handling_Level1,SCALER_Money_Per_Head,SCALER_Growth,SCALER_Credit_by_Score,SCALER_Determinstic,SCALER_Growth1,SCALER_Feedback1,SCALER_Previous_Claims_MEDIAN_Premium_Amount,SCALER_IsNull_Health_Score,SCALER_Previous_Claims_MEAN_Premium_Amount,SCALER_Previous_Claims,SCALER_Previous_Claims_STD_Premium_Amount,SCALER_IsNull_Customer_Feedback,SCALER_Previous_Claims_MAX_Premium_Amount,SCALER_Feedback3,SCALER_IsNull_Previous_Claims,SCALER_IsNull_Marital_Status,SCALER_Health_Score,SCALER_Health_Risk_Score,SCALER_Feedback2,SCALER_CreditInsurance,SCALER_Sin_Year,SCALER_IsNull_Credit_Score,SCALER_Health_Age_Interaction,SCALER_ENCODED_Policy_Start_Date_-_Year,SCALER_ENCODED_Policy_Start_Date_-_Quarter,SCALER_Feedback4,SCALER_IsNull_Number_of_Dependents,SCALER_IsNull_Occupation,SCALER_Health_Conscious_Level1,SCALER_Sin_Month,SCALER_Policy_Start_Date_-_Month,SCALER_Health_Conscious_Level,SCALER_Health_Conscious_Level_MEAN_Premium_Amount,SCALER_Health_Conscious_Level_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_MEAN_Premium_Amount,SCALER_Number_of_Dependents_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_STD_Premium_Amount,SCALER_Insurance_Duration_MEAN_Premium_Amount,SCALER_Insurance_Duration_MEDIAN_Premium_Amount,SCALER_Health_Conscious_Level_MAX_Premium_Amount,SCALER_Credit_Health_Score,SCALER_Occupation_MEDIAN_Premium_Amount,SCALER_Occupation_MAX_Premium_Amount,SCALER_Occupation_MEAN_Premium_Amount,SCALER_Previous_Claims_MIN_Premium_Amount,SCALER_Insurance_Duration_MAX_Premium_Amount,SCALER_ENCODED_Occupation_Self-Employed,SCALER_Age,SCALER_Insurance_Duration_STD_Premium_Amount
894006,5,-0.171872,0.806867,0.0,0.005925,-0.236990,-0.248774,0.037724,0.860841,0.308321,-0.202541,-0.309082,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,-0.25,1.0,0.0,-0.388978,0.388978,-0.357206,0.135327,0.0,0.0,-0.685754,0.333333,0.4,-0.509150,0.0,0.0,0.367600,7.347881e-16,-0.6,0.00000,0.2,0.000000,0.125,0.000000,0.032506,0.00,0.000000,0.002049,-0.333333,0.4,-0.619544,0.0,-0.333333,1.0,-0.913043,0.218186,0.000000
723021,1,0.063713,0.188841,0.0,0.147498,0.047326,0.077252,0.593180,-0.647249,0.430822,-0.167173,0.750902,52.0,0.0,57.590036,1.0,30.472002,0.0,-4.5,1.50,0.0,0.0,-0.331376,0.331376,1.223696,-0.802243,0.0,0.0,-0.526865,0.333333,0.6,0.547349,0.0,0.0,-0.504564,-7.347881e-16,0.6,-1.12816,0.0,0.631238,0.250,3.658057,-1.483870,-2.75,-2.166667,-0.168067,0.000000,0.0,0.000000,0.0,-0.666667,0.0,-0.652174,-0.132030,0.469401
845422,5,-0.444731,-0.695279,0.0,-0.507965,-0.305490,-0.396934,-0.449912,-1.216828,-0.304342,-0.089702,-0.428368,108.0,0.0,98.188795,2.0,33.769446,0.0,-4.5,0.25,0.0,0.0,0.719002,-0.719002,-0.666667,-0.388037,-1.0,0.0,0.135286,0.000000,0.0,-0.150839,0.0,0.0,0.812453,9.797174e-16,-0.8,0.00000,0.2,0.000000,0.125,0.000000,0.032506,0.00,0.000000,0.207198,0.000000,0.0,0.000000,0.0,-0.333333,0.0,-0.565217,0.218186,0.469401


#
---
#

# Base Model Building

In [22]:
# from xgboost import XGBRegressor
# from sklearn.metrics import root_mean_squared_log_error

In [23]:
# model = XGBRegressor(verbosity=3)

In [24]:
# model.fit(x_train, np.log(y_train))

In [25]:
# prediction = model.predict(x_validate)

In [26]:
# prediction[prediction < 0] = 0

In [27]:
# root_mean_squared_log_error(y_validate, prediction)

In [28]:
# prediction = pd.DataFrame({"id" : test.index, "Premium Amount" : model.predict(test)})
# prediction

In [29]:
# prediction.to_csv("Dileep's_Submission_Take3.csv", index=False)

#
---
#

# Hyper-tuning -- IT is taking too much time

In [30]:
# from sklearn.model_selection import RandomizedSearchCV

In [31]:
# esimator = XGBRegressor(n_jobs=-1, verbosity=3)

In [32]:
# param_grid = {
#     "n_estimators" : range(50, 1000),
#     "max_depth" : range(5, 30)
# }

In [33]:
# cv_model = RandomizedSearchCV(estimator=esimator, param_distributions=param_grid, verbose=3, n_jobs=-1, cv=2, n_iter=60)

In [34]:
# cv_model.fit(x_train, y_train)

#
---
#

# Optuna Hyper-parameter Tuning

In [35]:
def objective_lgbm(trial):
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 300, 1500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0),
        'device': 'gpu',
        'gpu_platform_id': 1,
        'gpu_device_id': 0,
        'verbose': -1,
    }

    model = LGBMRegressor(**params)
    model.fit(x_train, y_train, 
              eval_set=[(x_validate, y_validate)], 
              eval_metric='rmse', 
              callbacks=[early_stopping(stopping_rounds=200)])
    preds = model.predict(x_validate)
    return root_mean_squared_error(y_validate, preds)

# study_lgbm = optuna.create_study(direction='minimize')
# study_lgbm.optimize(objective_lgbm, n_trials=50)
# print("Best parameters for LightGBM:", study_lgbm.best_params)

In [36]:
study_lgbm.best_params

NameError: name 'study_lgbm' is not defined

In [ ]:
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0),
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor'
    }

    model = XGBRegressor(**params)
    model.fit(x_train, y_train, 
              eval_set=[(x_validate, y_validate)],
              verbose=3)
    preds = model.predict(x_validate)
    return root_mean_squared_error(y_validate, preds)

# study_xgb = optuna.create_study(direction='minimize')
# study_xgb.optimize(objective_xgb, n_trials=50)
# print("Best parameters for XGBoost:", study_xgb.best_params)

In [ ]:
study_xgb.best_params

#
---
#

# Let's do Cross-Validation

In [ ]:
X["SCALER_Previous_Claims_STD_Premium_Amount"].fillna(X["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [ ]:
test["SCALER_Previous_Claims_STD_Premium_Amount"].fillna(test["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [ ]:
lgb_params = {
    'n_estimators': 1305,
     'learning_rate': 0.057039638071732894,
     'num_leaves': 134,
     'max_depth': 11,
     'min_child_samples': 94,
     'subsample': 0.8818839454903746,
     'colsample_bytree': 0.8404645920435989,
     'reg_alpha': 2.2092334134699305,
     'reg_lambda': 6.978035716970699
 }

# cat_params = {
#     'iterations': 3000, 
#     'learning_rate': 0.038365175314273574, 
#     'depth': 11, 
#     'l2_leaf_reg': 3.596285147607088, 
#     'bagging_temperature': 0.2618728648567565
# }

xgb_params = {
    'n_estimators': 265,
     'learning_rate': 0.015659722355698517,
     'max_depth': 12,
     'min_child_weight': 8,
     'subsample': 0.874124890763571,
     'colsample_bytree': 0.7149043184973538,
     'reg_alpha': 7.485038259722735,
     'reg_lambda': 6.978704847907894
}




In [ ]:
from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error, r2_score


splitter = KFold(n_splits=10)
models = []


count = 1

folds_df = pd.DataFrame()

for train_index, test_index in splitter.split(X, Y_log):
    fold_xtrain  = X.iloc[train_index, :]
    fold_ytrain  = Y_log.iloc[train_index]
    fold_xtest  = X.iloc[test_index, :]
    fold_ytest  = Y_log.iloc[test_index]

    # LightGBM model
    lgbm_model = LGBMRegressor(**lgb_params, device='gpu', gpu_platform_id=1, gpu_device_id=0, verbose=3)

    # CatBoost model
    # cat_model = CatBoostRegressor(**cat_params, verbose=3, task_type= 'GPU')

    # XGBoost model
    xgb_model = XGBRegressor(**xgb_params, verbose=3, tree_method="gpu_hist", predictor= 'gpu_predictor')

    # Voting Regressor
    model = VotingRegressor(
        estimators=[
            ('lgbm', lgbm_model),
            # ('cat', cat_model),
            ('xgb', xgb_model)
        ]
    )

    model.fit(fold_xtrain, fold_ytrain)
    fold_pred = np.maximum(0, model.predict(fold_xtest))
    fold_rmsle = root_mean_squared_log_error(np.expm1(fold_ytest), np.expm1(fold_pred))
    print(f"Fold {count} RMSLE: {fold_rmsle}")
    models.append(model)

    folds_df[f"Fold {count}"] = model.predict(test)
    count += 1

In [ ]:
# splitter = KFold(n_splits=10)

In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor
# from xgboost import XGBRegressor
# from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error, r2_score
# gb_model = GradientBoostingRegressor(verbose=3, loss="absolute_error", learning_rate=0.5, n_estimators=150)
# xg_model = XGBRegressor(verbose=3, loss="absolute_error", learning_rate=0.075, n_estimators=100)

# count = 1

# folds_df = pd.DataFrame()

# for train_index, test_index in splitter.split(X, Y_log):
#     fold_xtrain  = X.iloc[train_index, :]
#     fold_ytrain  = Y_log.iloc[train_index]
#     fold_xtest  = X.iloc[test_index, :]
#     fold_ytest  = Y_log.iloc[test_index]

#     # print(fold_xtrain.shape, fold_ytrain.shape, fold_xtest.shape, fold_ytest.shape)

#     fold_ypred = xg_model.fit(fold_xtrain, fold_ytrain).predict(fold_xtest)
#     print(f"The Root mean squared log error for FOLD - {count} is --> {root_mean_squared_log_error(fold_ytest, fold_ypred)}")
    
#     folds_df[f"Fold {count}"] = xg_model.predict(test)
#     count += 1

In [ ]:
with open('models.pkl', 'wb') as file:
    pickle.dump(models, file)

print("Cross-validation models saved!")

In [ ]:
# from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error, r2_score

# print(f"The Root mean squared log error is --> {np.exp(root_mean_squared_log_error(fold_ytest, fold_ypred))}")
# print(f"The R-Square is --> {r2_score(fold_ytest, fold_ypred)}")


In [ ]:
folds_df.to_csv("folds_prediction.csv", index=False)

#
---
#

In [ ]:
preds = pd.read_csv("folds_prediction.csv")
preds

In [ ]:
pred = preds.mean(axis=1).apply(np.expm1)

In [ ]:
prediction = pd.DataFrame({"id" : test.index, "Premium Amount" : pred})
prediction

In [ ]:
prediction.to_csv("Dileep's_Submission_Take27.csv", index=False)

In [60]:
# Take 20 did after Feature engineering
# Before -- 821 Rank
# After -- 757

#
---
#

# Lets implement... `STACKING`

In [106]:
X.fillna(X["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [107]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, IsolationForest
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_log_error

In [111]:
lir_model = LinearRegression()
knn_model = KNeighborsRegressor()
rf_model = RandomForestRegressor(verbose=3, n_estimators=40)
gb_model = GradientBoostingRegressor(verbose=3, loss="absolute_error", learning_rate=0.5, n_estimators=250)
xgb_model = XGBRegressor(verbose=3)
ridge_model = Ridge()
if_model = IsolationForest(verbose=3)

In [112]:
stacked_model = StackingRegressor(estimators=[
                    ("rf_model", rf_model),
                    # ("knn_model", knn_model),
                    ("xgb_model", xgb_model),
                    ("gb_model", gb_model),
                    # ("lir_model", lir_model)
                ], final_estimator=lir_model, verbose=3)

In [113]:
gb_model.fit(X, np.log(Y))

      Iter       Train Loss   Remaining Time 
         1           0.7717           34.10m
         2           0.7642           32.46m
         3           0.7623           30.65m
         4           0.7610           30.38m
         5           0.7604           30.08m
         6           0.7600           29.70m
         7           0.7598           29.28m
         8           0.7594           29.04m
         9           0.7592           28.73m
        10           0.7591           28.47m
        11           0.7591           28.43m
        12           0.7590           28.31m
        13           0.7589           28.50m
        14           0.7589           28.71m
        15           0.7588           28.62m
        16           0.7587           28.36m
        17           0.7587           28.18m
        18           0.7586           28.15m
        19           0.7584           27.90m
        20           0.7582           28.12m
        21           0.7582           28.06m
        2

GradientBoostingRegressor(learning_rate=0.5, loss='absolute_error',
                          n_estimators=250, verbose=3)

In [114]:
prediction = gb_model.predict(x_validate)

In [115]:
# prediction[prediction < 0] = prediction.mean()

In [116]:
root_mean_squared_log_error(y_validate, prediction)

np.float64(4.67300233947815)

In [122]:
test.fillna(test["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [123]:
prediction = pd.DataFrame({"id" : test.index, "Premium Amount" : np.exp(gb_model.predict(test))})
prediction

,id,Premium Amount
0,1200000,957.075734
1,1200001,920.077180
2,1200002,905.662536
3,1200003,905.194894
4,1200004,833.240238
...,...,...
799995,1999995,1036.266459
799996,1999996,903.047919
799997,1999997,1005.565005
799998,1999998,935.444136


In [124]:
prediction.to_csv("Dileep's_Submission_Take14.csv", index=False)